In [ ]:
%pylab inline

# SQLAlchemy

# ~~object-relational mapper~~

# RDBMS toolbox

# RDBMS were so last year...

# last year

# ddlgenerator

## Set up [data](http://github.com/catherinedevlin/pycon2015_sqla_lightning/data/provinces.yaml).

In [ ]:
!ddlgenerator -i postgresql data/provinces.yaml
# !ddlgenerator -i postgresql http://github.com/catherinedevlin/pycon2015_sqla_lightning/data/provinces.yaml

In [ ]:
!dropdb pycon
!createdb pycon
!ddlgenerator -i postgresql data/provinces.yaml | psql pycon

# ipython_sql

## Access the data

In [ ]:
%load_ext sql

In [ ]:
%sql postgresql://:@/pycon

In [ ]:
%sql select * from provinces

In [ ]:
prov = %sql select * from provinces
prov[1].name 

In [ ]:
p = prov.DataFrame()

In [ ]:
p

In [ ]:
p.plot()

# rdbms-subsetter

## Get a consistent test database.

In [ ]:
!dropdb pycon_test
!createdb pycon_test
!pg_dump --schema-only pycon | psql pycon_test

In [ ]:
!rdbms-subsetter -y postgresql://:@/pycon postgresql://:@/pycon_test 0.4

In [ ]:
%sql postgresql://:@/pycon_test SELECT * FROM provinces

In [ ]:
%sql SELECT * FROM cities

# Still needed

* normalization/denormalization toolkit
* Jupyter Notebooks for tuning & monitoring
* inter-RDBMS translator

Catherine Devlin

http://tinyurl.com/pycon2015-sqla-tools